# CatBoostClassifier

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 2000) # since we have a lot of features
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("data"))

['test.csv', 'train.csv', 'sample_submission.csv']


In [2]:
''' Import data '''

DATA_DIR = 'data'
train_file = os.path.join(DATA_DIR, 'train.csv')
test_file = os.path.join(DATA_DIR, 'test.csv')
submission_file = os.path.join(DATA_DIR, 'sample_submission.csv')

train = pd.read_csv(train_file)
test = pd.read_csv(test_file)

X = train.drop(columns=['y', 'sample_id'])
y = train.y
X_test = test.drop(columns=['sample_id'])

X_all = X.append(X_test)

In [3]:
''' Gather all preprocessing functions '''

from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import bisect
from scipy.stats import shapiro
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM


''' 1 - divide to training and holdout datasets '''

X_train, X_hold, y_train, y_hold = train_test_split(X, y, random_state=3, test_size=0.3)

''' 2 - define the same validation scheme for all models '''

skf = RepeatedStratifiedKFold(n_repeats=5, random_state=17)


''' Function 1 - delete all columns with NaN or Inf '''
columns_to_keep = X_all.replace([np.inf, -np.inf], np.nan).dropna(axis=1).columns

def del_nan(x):
    return x[columns_to_keep]
delete_nan = FunctionTransformer(del_nan, validate=False)


''' Function 2 - delete NaN-only columns '''

na_cols = X_all[X_all.isna().any()[lambda x: x].index].isna().agg(['sum', 'count']).T

all_na_cols = na_cols[na_cols['sum'] == na_cols['count']]

def drop_all_na(X):
    ''' Here we drop all columns which consist of NaN only '''
    idx = all_na_cols.index.values
    return X.drop(columns=idx)

drop_all_na_f = FunctionTransformer(drop_all_na, validate=False)


''' Function 3 - replace NaN '''

has_na_cols = na_cols.T.drop(columns=all_na_cols.index.values).T
has_na_cols['perc'] = has_na_cols['sum']/has_na_cols['count']

'''def replace_na(X, add_boolean=True, threshold=0.3):
    Here we apply two filling strategies
            for two types of columns with na:
            % na > threshold - fill with mean
            % na < threshold - fill with 0.
        Plus we add additional columns showing
            if the value was na before transformation.
    X_copy = X.copy()
    cols_less = has_na_cols[has_na_cols['perc'] < threshold].index.values
    cols_more = has_na_cols[has_na_cols['perc'] >= threshold].index.values
    X_less = X_copy[cols_less]
    X_less = X_less.fillna(X_less.mean())
    X_copy[cols_less] = X_less
    X_copy[cols_more] = X_more
    if add_boolean:
        was_na = X[has_na_cols.index.values].isna().astype('int64')
        was_na.columns = was_na.columns.map(lambda x: x + '_na')
        X_copy = pd.concat([X_copy, was_na], axis=1)
    return X_copy'''
    
# I should have checked columns for categorical features before

def replace_na(X, add_boolean=True):
    ''' Here we apply mean filling strategy.
        And add additional columns showing if the values
        was na before transformation.
    '''
    X_copy = X.copy()
    cols = has_na_cols.index.values
    X_c = X_copy[cols]
    X_c = X_c.fillna(X_all.replace([np.inf, -np.inf], np.nan).mean())
    X_copy[cols] = X_c
    if add_boolean:
        was_na = X[has_na_cols.index.values].isna().astype('int64')
        was_na.columns = was_na.columns.map(lambda x: x + '_na')
        X_copy = pd.concat([X_copy, was_na], axis=1)
    return X_copy

replace_na_f = FunctionTransformer(replace_na, validate=False)


''' Function 4 - replace inf '''

pos_inf_cols = X_all.fillna(0).replace([np.inf], 
                                       np.nan).isna().any()[lambda x: x].index
pos_inf_cols = X_all[pos_inf_cols].replace([np.inf], 
                                           np.nan).isna().agg(['sum', 'count']).T
neg_inf_cols = X_all.fillna(0).replace([-np.inf],
                                       np.nan).isna().any()[lambda x: x].index

def fill_inf(X, add_boolean=True, multiplier=2):
    '''
    Here we apply filling strategy for infinite values:
        we replace it with absolute maximum value * multiplier.
    Additionally, we new columns showing if the value
        was infinite before filling.
    '''
    X_copy = X.copy()
    X_pos = X_copy[pos_inf_cols.index.values]
    to_fill = X_pos.replace(np.inf, np.nan).abs().max() * multiplier + 1.0
    X_pos = X_pos.replace(np.inf, np.nan).fillna(to_fill)
    X_copy[pos_inf_cols.index.values] = X_pos
    if add_boolean:
        was_inf = X[pos_inf_cols.index.values].replace(np.inf, np.nan).isna().astype('int64')
        was_inf.columns = was_inf.columns.map(lambda x: x + '_inf')
        X_copy = pd.concat([X_copy, was_inf], axis=1)
    return X_copy

fill_inf_f = FunctionTransformer(fill_inf, validate=False)


''' First block of data cleaning is gathered in one function '''

def pre_proc(X):
    return fill_inf(replace_na(drop_all_na(X)))


''' Function and class 5 - delete zero-variance features '''

w = fill_inf(replace_na(drop_all_na(X_all))).var()
zero_var_cols = w[np.isclose(w, 0)]

def drop_zero_var(X):
    idx = zero_var_cols.index.values
    return X.drop(columns=idx)

drop_zero_var_f = FunctionTransformer(drop_zero_var)

class Drop_zero_var:
    def __init__(self):
        self.zero_var_cols = []
        self.is_fitted = False
        
    def fit(self, X, y=None):
        w = X.var()
        self.zero_var_cols = w[np.isclose(w, 0.0)].index.values
        self.is_fitted = True
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        return X_copy.drop(columns=self.zero_var_cols)
    
    def fit_transform(self, X, y=None):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)

print(Drop_zero_var().fit_transform(pre_proc(X_train), y_train).shape)

''' Function and class 6 - drop highly correlated features '''

'''
# Here is the function
print(drop_zero_var(pre_proc(X_all)).shape)
corr_matrix = pre_proc(X_all).corr().abs()
print(corr_matrix.shape)

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(len(to_drop))

'''

class Drop_corr:
    def __init__(self, threshold=0.95):
        self.to_drop = []
        self.is_fitted = False
        self.threshold = threshold
        
    def fit(self, X, y=None):
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        self.to_drop = [column for column in upper.columns if any(upper[column] > self.threshold)]
        self.is_fitted = True
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        return X_copy.drop(columns=self.to_drop)
    
    def fit_transform(self, X, y=None):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)

print(Drop_corr().fit_transform(pre_proc(X_train), y_train).shape)
    
''' Class 7 - encode categorical labels '''

class LE_df:
    def __init__(self, threshold=5):
        self.le_dict = {}
        self.threshold = threshold
        
    def fit(self, X, y=None):
        X_copy = X.copy()
        counts = X_copy.apply(lambda x: x.value_counts().shape[0])
        categorical = X_copy[counts[counts <= self.threshold].index]
        for col in categorical.columns:
            le = LabelEncoder()
            le.fit(categorical[col])
            self.le_dict[col] = le
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        for col in self.le_dict.keys():
            X_copy[col] = X_copy[col].map(lambda s: -100500 if s not in \
                                          self.le_dict[col].classes_ else s)
            if np.any(X_copy[col]==-100500):
                le_classes = self.le_dict[col].classes_.tolist()
                le_classes.append(-100500)
                self.le_dict[col].classes_ = np.asarray(le_classes)
            X_copy[col] = self.le_dict[col].transform(X_copy[col])
        return X_copy
    
    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

print(LE_df().fit_transform(pre_proc(X_train), y_train).shape)
    
''' Class 8 - target mean encode categorical labels '''

class LOOTME_df:
    ''' Class performs leave-one-out target mean encoding '''
    def __init__(self, threshold=5):
        self.threshold = threshold
        self.tme_dict = {}
        self.is_fitted = False
        
    def fit(self, X, y):
        X_copy = X.copy()
        y_copy = y.copy()
        counts = X_copy.apply(lambda x: x.value_counts().shape[0])
        categorical = X_copy[counts[counts <= self.threshold].index]
        for col in categorical.columns:
            df = pd.concat((X_copy[col], y_copy), axis=1)
            df.columns = ['col', 'y']
            tme = {}
            for value in df['col'].unique():
                y_val = df[df['col'] == value]['y'].values
                col_val = df[df['col'] == value]['col'].values
                loo_mean = np.mean((np.full_like(col_val, y_val.sum()) - y_val) / y_val.shape[0])
                tme[value] = loo_mean
            self.tme_dict[col] = tme
        self.is_fitted = True
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        for col in self.tme_dict.keys():
            X_copy[col] = X_copy[col].map(lambda s: -100500 if s not in self.tme_dict[col].keys() else s)
            self.tme_dict[col][-100500] = 0.0
            X_copy[col] = X_copy[col].map(self.tme_dict[col])
        return X_copy
        
    
    def fit_transform(self, X, y):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)
    
print(LOOTME_df().fit_transform(pre_proc(X_train), y_train).shape)


''' Class 9 - One Hot Encoding of categorical features '''

class OneHot_df:
    def __init__(self, threshold=5, drop_collinear=False):
        self.threshold = threshold
        self.drop_collinear = drop_collinear
        self.categorical = []
        self.ohe = None
        self.column_names = None
        self.is_fitted = False
        
    def fit(self, X, y=None):
        X_copy = X.copy()
        counts = X_copy.apply(lambda x: x.value_counts().shape[0])
        X_cat = X_copy[counts[(counts <= self.threshold) & (counts > 1)].index]
        ohe = OneHotEncoder(handle_unknown='ignore')
        ohe.fit(X_cat)
        self.categorical = X_cat.columns
        self.ohe = ohe
        self.is_fitted = True
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_cat = X_copy.copy()[self.categorical]
        X_cat_tr = self.ohe.transform(X_cat).toarray()
        if not self.column_names:
            column_names = []
            for i, col in enumerate(self.categorical):
                column_names.extend([col + '_' + str(s) for s in self.ohe.categories_[i]])
            self.column_names = column_names
        X_cat_tr = pd.DataFrame(X_cat_tr, columns=self.column_names, index=X_copy.index)
        X_other = X_copy.drop(columns=self.categorical)
        X_copy = pd.concat([X_other, X_cat_tr], axis=1)
        if self.drop_collinear:
            to_drop = pd.Series(self.column_names, 
        index=self.column_names).apply(lambda x: x.endswith('_0'))[lambda x: x].index
            X_copy.drop(columns=to_drop)
        return X_copy
    
    def fit_transform(self, X, y=None):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)

print(OneHot_df().fit(pre_proc(X_train)).transform(pre_proc(X_hold)).shape)

''' Class 10 - find best normalizing transformation '''

def get_shapiro_p(X):
    _, p = shapiro(X)
    return p

class Find_Trans:
    def __init__(self, threshold=10):
        self.threshold = threshold
        self.pvals = []
        self.numerical = []
        self.is_fitted = False
        self.mms = MinMaxScaler()
        self.trans_dict = {'no': lambda x: x,
                           'x^2': lambda x: np.power(x, 2), 
                           'x^3': lambda x: np.power(x, 3), 
                           'log(x)': lambda x: np.log(x), 
                           #'exp(x)': lambda x: np.exp(x/10), 
                           'sqrt(x)': lambda x: np.sqrt(x)}
    
    def fit(self, X, y=None):
        X_copy = X.copy()
        counts = X_copy.apply(lambda x: x.value_counts().shape[0])
        self.numerical = counts[counts > self.threshold].index.values
        X_numer = X_copy[self.numerical]
        idx = X_numer.index
        X_numer = pd.DataFrame(self.mms.fit_transform(X_numer) + 0.01,
                               columns=self.numerical,
                               index=idx)
        pvals = pd.DataFrame(X_numer.apply(get_shapiro_p), columns=['no'])
        for name in self.trans_dict.keys():
            if name not in pvals.columns:
                pvals[name] = X_numer.apply(self.trans_dict[name]).apply(get_shapiro_p)
        self.pvals = pvals
        self.is_fitted = True
        return self
    
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_numer = X_copy[self.numerical]
        idx = X_numer.index
        X_numer = pd.DataFrame(self.mms.transform(X_numer) + 0.01,
                               columns=self.numerical,
                               index=idx)
        best_transform = self.pvals.idxmax(axis=1)
        best_transform = best_transform.map(self.trans_dict).to_dict()
        X_numer = X_numer.apply(best_transform)
        X_numer = X_numer.fillna(X_numer.min()-1)
        for col in self.numerical:
            X_copy[col] = X_numer[col]
        return X_copy
    
    def fit_transform(self, X, y=None):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)

print(Find_Trans().fit(pre_proc(X_train), y_train).transform(pre_proc(X_hold)).shape)


''' Class 11 - find and drop outliers '''

class DetectOut_df:
    def __init__(self, method='IsolationForest', out_fraction=0.05, 
                 drop=True, random_state=17,
                 kernel='rbf', shrink=True):
        self.method = method
        self.out_fraction = out_fraction
        self.drop = drop
        self.is_fitted = False
        self.random_state = random_state
        self.kernel = kernel
        self.shrink = shrink
        self.model = None
        self.train_set = []
        
    def fit(self, X, y=None):
        if self.method == 'IsolationForest':
            model = IsolationForest(random_state=self.random_state,
                        n_estimators=100,
                        max_features=0.9,
                        contamination=self.out_fraction, #
                        n_jobs=-1,
                        behaviour='new')
        if self.method == 'OneClassSVM':
            model = OneClassSVM(kernel=self.kernel,
                     nu=self.out_fraction,
                     shrinking=self.shrink,
                     gamma='scale'
                    )
        model.fit(X, y)
        self.model = model
        self.is_fitted = True
        self.train_set = X.copy()
        return self
    
    def transform(self, X, y):
        ''' We shouldn't predict any outliers for test set '''
        X_copy = X.copy()
        X_copy['outlier'] = 1
        X_copy['y'] = y.copy()
        if self.train_set.equals(X):
            X_copy['outlier'] = self.model.predict(X)
        if self.drop:
            X_copy = X_copy[X_copy['outlier'] == 1]
            X_copy = X_copy.drop(columns=['outlier'])
        y_copy = X_copy['y']
        X_copy = X_copy.drop(columns=['y'])
        return X_copy, y_copy
    
    def fit_transform(self, X, y):
        if not self.is_fitted:
            self.fit(X, y)
        return self.transform(X, y)
    
print(DetectOut_df().fit_transform(pre_proc(X_train), y_train)[0].shape)

(766, 1777)
(766, 932)
(766, 1908)
(766, 1908)
(329, 2518)
(329, 1908)
(727, 1908)


# Базовый CatBoost

In [6]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

cat = CatBoostClassifier(learning_rate=0.001, 
                                                iterations=500,
                                                loss_function='CrossEntropy',
                                                custom_metric='AUC:hints=skip_train~false',
                                                eval_metric='AUC:hints=skip_train~false',
                                                random_state=17,
                                                rsm=0.1,
                                                bootstrap_type='Bernoulli',
                                                subsample=0.9,
                                                verbose=100)
pipe_cat = Pipeline([('drop_na', delete_nan),
                     ('cat', cat)])

pipe_cat.fit(X_train, y_train)

#scores = cross_val_score(pipe_cat, X_train, y_train, cv=skf, scoring='roc_auc')

print('Train ROC AUC: %.4f' % roc_auc_score(y_train, pipe_cat.predict_proba(X_train)[:,1]))
#print('Cross-validation ROC AUC: mean %.4f, std %.4f' % (scores.mean(), scores.std()))
print('Holdout ROC AUC: %.4f' % roc_auc_score(y_hold, pipe_cat.predict_proba(X_hold)[:,1]))

0:	learn: 0.8060754	total: 30.2ms	remaining: 15.1s
100:	learn: 0.8924454	total: 3.38s	remaining: 13.4s
200:	learn: 0.8949478	total: 6.55s	remaining: 9.75s
300:	learn: 0.8966257	total: 9.98s	remaining: 6.6s
400:	learn: 0.8992141	total: 13s	remaining: 3.21s
499:	learn: 0.9012720	total: 16.1s	remaining: 0us
Train ROC AUC: 0.9013
Holdout ROC AUC: 0.8191


## После трансформации фич

In [9]:
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

CORR_THRESHOLD = 0.94
CAT_THRESHOLD = 5 # <- and less unique values make feature categorical

cat = CatBoostClassifier(learning_rate=0.001, 
                                                iterations=500,
                                                loss_function='CrossEntropy',
                                                custom_metric='AUC:hints=skip_train~false',
                                                eval_metric='AUC:hints=skip_train~false',
                                                random_state=17,
                                                rsm=0.1,
                                                bootstrap_type='Bernoulli',
                                                subsample=0.9,
                                                verbose=100)
pipe_cat = Pipeline([('pre_proc', FunctionTransformer(pre_proc, validate=False)),
                      ('drop_zero_var', Drop_zero_var()),
                      ('drop_corr', Drop_corr(threshold=CORR_THRESHOLD)),
                      ('lootme', LOOTME_df(threshold=CAT_THRESHOLD)),
                      ('transform', Find_Trans(threshold=CAT_THRESHOLD)),
                     ('cat', cat)])

pipe_cat.fit(X_train, y_train)

#scores = cross_val_score(pipe_cat, X_train, y_train, cv=skf, scoring='roc_auc')

print('Train ROC AUC: %.4f' % roc_auc_score(y_train, pipe_cat.predict_proba(X_train)[:,1]))
#print('Cross-validation ROC AUC: mean %.4f, std %.4f' % (scores.mean(), scores.std()))
print('Holdout ROC AUC: %.4f' % roc_auc_score(y_hold, pipe_cat.predict_proba(X_hold)[:,1]))

0:	learn: 0.7889527	total: 27ms	remaining: 13.5s
100:	learn: 0.8905381	total: 2.92s	remaining: 11.6s
200:	learn: 0.8959516	total: 6.2s	remaining: 9.22s
300:	learn: 0.8985831	total: 8.78s	remaining: 5.8s
400:	learn: 0.9017381	total: 11.4s	remaining: 2.81s
499:	learn: 0.9048930	total: 14s	remaining: 0us
Train ROC AUC: 0.9049
Holdout ROC AUC: 0.8198


In [10]:
pipe_cat.fit(X, y)

submission = pd.read_csv(submission_file)
submission['y'] = pipe_cat.predict_proba(X_test)[:,1]
submission.to_csv('submission_cat_transform.csv', index=False)

0:	learn: 0.7877042	total: 26.2ms	remaining: 13.1s
100:	learn: 0.8896817	total: 3.84s	remaining: 15.2s
200:	learn: 0.8913231	total: 6.82s	remaining: 10.2s
300:	learn: 0.8931157	total: 10.2s	remaining: 6.75s
400:	learn: 0.8954426	total: 13s	remaining: 3.2s
499:	learn: 0.8980190	total: 16.4s	remaining: 0us


## Финальная модель - 0.88414

In [11]:
%%time
''' Let's finally play with Catboost '''

from sklearn.feature_selection import SelectKBest, f_classif
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

CORR_THRESHOLD = 0.94
CAT_THRESHOLD = 5 # <- and less unique values make feature categorical
#OUT_FRACTION = 0.05 # <- percent of outliers
#K = 400
L2_LEAF_REG = 0.5 # <- regularization coefficient for CatBoost
DEPTH = 10 # <- max tree depth for Catboost, can't be >10, or kernel will just die

cat = CatBoostClassifier(learning_rate=0.001, 
                         iterations=500,
                         loss_function='CrossEntropy',
                         l2_leaf_reg=L2_LEAF_REG,
                         depth=DEPTH,
                         custom_metric='AUC:hints=skip_train~false', #CPU-only
                         eval_metric='AUC:hints=skip_train~false', #CPU-only
                         random_state=17,
                         rsm=0.1, #CPU-only
                         bootstrap_type='Bernoulli',
                         subsample=0.9,
                         #task_type='GPU',
                         verbose=100)
pipe_cat = Pipeline([('pre_proc', FunctionTransformer(pre_proc, validate=False)),
                      ('drop_zero_var', Drop_zero_var()),
                      ('drop_corr', Drop_corr(threshold=CORR_THRESHOLD)),
                      ('lootme', LOOTME_df(threshold=CAT_THRESHOLD)),
                      ('transform', Find_Trans(threshold=CAT_THRESHOLD)),
                      #('feat_sel', SelectKBest(f_classif, K)),
                      ('cat', cat)])

pipe_cat.fit(X_train, y_train)

#scores = cross_val_score(pipe_cat, X_train, y_train, cv=skf, scoring='roc_auc')

print('Train ROC AUC: %.4f' % roc_auc_score(y_train, pipe_cat.predict_proba(X_train)[:,1]))
#print('Cross-validation ROC AUC: mean %.4f, std %.4f' % (scores.mean(), scores.std()))
print('Holdout ROC AUC: %.4f' % roc_auc_score(y_hold, pipe_cat.predict_proba(X_hold)[:,1]))

0:	learn: 0.8183259	total: 278ms	remaining: 2m 18s
100:	learn: 0.9601761	total: 33.6s	remaining: 2m 12s
200:	learn: 0.9649228	total: 1m	remaining: 1m 29s
300:	learn: 0.9673679	total: 1m 31s	remaining: 1m
400:	learn: 0.9700640	total: 2m 12s	remaining: 32.6s
499:	learn: 0.9722581	total: 2m 53s	remaining: 0us
Train ROC AUC: 0.9723
Holdout ROC AUC: 0.8433
CPU times: user 7min 27s, sys: 38.3 s, total: 8min 5s
Wall time: 3min 11s


In [12]:
pipe_cat.fit(X, y)

submission = pd.read_csv(submission_file)
submission['y'] = pipe_cat.predict_proba(X_test)[:,1]
submission.to_csv('submission_cat_modified.csv', index=False)

0:	learn: 0.8581760	total: 400ms	remaining: 3m 19s
100:	learn: 0.9522713	total: 42.5s	remaining: 2m 47s
200:	learn: 0.9557651	total: 1m 28s	remaining: 2m 10s
300:	learn: 0.9581447	total: 2m 11s	remaining: 1m 27s
400:	learn: 0.9607246	total: 2m 53s	remaining: 42.9s
499:	learn: 0.9631394	total: 3m 27s	remaining: 0us
